In [21]:
import numpy as np
import string
import pandas as pd
import nltk
import keras

from sklearn import random_projection
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords

from keras.models import Sequential
from keras.layers import Embedding, Dense, Dropout, LSTM
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D
from keras.optimizers import SGD
from keras import metrics

stop_words = set(stopwords.words('english') + list(string.punctuation))


# -------------- Helper Functions --------------
def tokenize(text):
    '''
    :param text: a doc with multiple sentences, type: str
    return a word list, type: list
    https://textminingonline.com/dive-into-nltk-part-ii-sentence-tokenize-and-word-tokenize
    e.g.
    Input: 'It is a nice day. I am happy.'
    Output: ['it', 'is', 'a', 'nice', 'day', 'i', 'am', 'happy']
    '''
    tokens = []
    for word in nltk.word_tokenize(text):
        word = word.lower()
        if word not in stop_words and not word.isnumeric():
            tokens.append(word)
    return tokens


def get_sequence(data, seq_length, vocab_dict):
    '''
    :param data: a list of words, type: list
    :param seq_length: the length of sequences,, type: int
    :param vocab_dict: a dict from words to indices, type: dict
    return a dense sequence matrix whose elements are indices of words,
    '''
    data_matrix = np.zeros((len(data), seq_length), dtype=int)
    for i, doc in enumerate(data):
        for j, word in enumerate(doc):
            # YOUR CODE HERE
            if j == seq_length:
                break
            word_idx = vocab_dict.get(word, 1) # 1 means the unknown word
            data_matrix[i, j] = word_idx
    return data_matrix


def read_data(file_name, input_length, vocab=None):
    """
    https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html
    """
    df = pd.read_csv(file_name)
    df['words'] = df['text'].apply(tokenize)

    if vocab is None:
        vocab = set()
        for i in range(len(df)):
            for word in df.iloc[i]['words']:
                vocab.add(word)
    vocab_dict = dict()
    vocab_dict['<pad>'] = 0 # 0 means the padding signal
    vocab_dict['<unk>'] = 1 # 1 means the unknown word
    vocab_size = 2
    for v in vocab:
        vocab_dict[v] = vocab_size
        vocab_size += 1

    data_matrix = get_sequence(df['words'], input_length, vocab_dict)
    stars = df['stars'].apply(int) - 1
    return df['review_id'], stars, data_matrix, vocab
# ----------------- End of Helper Functions-----------------


def load_data(input_length):
     # Load training data and vocab
    train_id_list, train_data_label, train_data_matrix, vocab = read_data("data/train.csv", input_length)
    K = max(train_data_label)+1  # labels begin with 0

    # Load testing data
    test_id_list, _, test_data_matrix, _ = read_data("data/test.csv", input_length, vocab=vocab)
    test_data_label = pd.read_csv("data/valid.csv")['stars'] - 1
    
    print("Vocabulary Size:", len(vocab))
    print("Training Set Size:", len(train_id_list))
    print("Test Set Size:", len(test_id_list))
    print("Training Set Shape:", train_data_matrix.shape)
    print("Testing Set Shape:", test_data_matrix.shape)

    # Converts a class vector to binary class matrix.
    # https://keras.io/utils/#to_categorical
    train_data_label = keras.utils.to_categorical(train_data_label, num_classes=K)
    test_data_label = keras.utils.to_categorical(test_data_label, num_classes=K)
    return train_data_matrix, train_data_label, test_data_matrix, test_data_label, vocab

In [6]:
test_data = pd.read_csv("data/valid.csv")
test_data_label = pd.read_csv("data/valid.csv")['stars'] - 1

In [8]:
# Hyperparameters
input_length = 30
embedding_size = 100
hidden_size = 100
batch_size = 100
dropout_rate = 0.5
learning_rate = 0.1
total_epoch = 10


In [16]:
train_data_matrix, train_data_label, test_data_matrix, test_data_label, vocab = load_data(input_length)

Vocabulary Size: 114655
Training Set Size: 100000
Test Set Size: 10000
Training Set Shape: (100000, 30)
Testing Set Shape: (10000, 30)


In [24]:
# Data shape
N = train_data_matrix.shape[0]
K = train_data_label.shape[1]

input_size = len(vocab) + 2
output_size = K

In [27]:
# New model
model = Sequential()
        
    
# CNN
model.add(Conv2D(filters=20, kernel_size=(5,5), padding='valid', input_shape=K, activation='tanh'))
# First pooling layer
model.add(MaxPool2D(pool_size=(2,2)))
# Second convolutional layer
model.add(Conv2D(filters=16, kernel_size=(5,5), padding='valid', activation='tanh'))
# Second pooling layer 
model.add(MaxPool2D(pool_size=(2,2)))
# Flatten the 2D layer into 1D
model.add(Flatten())

# Fully-connected layer
model.add(Dense(120, activation='tanh'))
model.add(Dense(84, activation='tanh'))

# Output 5 labels
model.add(Dense(5, activation='softmax'))



TypeError: 'int' object is not iterable

In [ ]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# training
model.fit(train_data_matrix, train_data_label, epochs=total_epoch, batch_size=batch_size)
# testing
train_score = model.evaluate(train_data_matrix, train_data_label, batch_size=batch_size)
test_score = model.evaluate(test_data_matrix, test_data_label, batch_size=batch_size)

print('Training Loss: {}\n Training Accuracy: {}\n'
      'Testng Loss: {}\n Testing accuracy: {}'.format(
          train_score[0], train_score[1],
          test_score[0], test_score[1]))



In [29]:
# New model
model = Sequential()
        
    
# CNN
model.add(Conv2D(filters=20, kernel_size=(5,5), padding='valid', input_shape=(28,28, 1), activation='tanh'))
# First pooling layer
model.add(MaxPool2D(pool_size=(2,2)))
# Second convolutional layer
model.add(Conv2D(filters=16, kernel_size=(5,5), padding='valid', activation='tanh'))
# Second pooling layer 
model.add(MaxPool2D(pool_size=(2,2)))
# Flatten the 2D layer into 1D
model.add(Flatten())


# LSTM
model = Sequential()
# embedding layer and dropout
model.add(Embedding(input_dim=input_size, output_dim=embedding_size, input_length=input_length))
model.add(Dropout(dropout_rate))

model.add(TimeDistributed(cnn, ...))
model.add(LSTM(units=hidden_size))

# output layer
model.add(Dense(K, activation='softmax'))

# SGD optimizer with momentum
optimizer = SGD(lr=learning_rate, decay=1e-6, momentum=0.9, nesterov=True)

# compile model
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# training
model.fit(train_data_matrix, train_data_label, epochs=total_epoch, batch_size=batch_size)
# testing
train_score = model.evaluate(train_data_matrix, train_data_label, batch_size=batch_size)
test_score = model.evaluate(test_data_matrix, test_data_label, batch_size=batch_size)

print('Training Loss: {}\n Training Accuracy: {}\n'
      'Testng Loss: {}\n Testing accuracy: {}'.format(
          train_score[0], train_score[1],
          test_score[0], test_score[1]))



Epoch 1/10
100000/100000 [==============================] - 106s 1ms/step - loss: 1.3231 - acc: 0.4732
Epoch 2/10
100000/100000 [==============================] - 103s 1ms/step - loss: 1.0079 - acc: 0.5828
Epoch 3/10
100000/100000 [==============================] - 98s 981us/step - loss: 0.9426 - acc: 0.6082
Epoch 4/10
100000/100000 [==============================] - 98s 976us/step - loss: 0.9078 - acc: 0.6201
Epoch 5/10
100000/100000 [==============================] - 94s 937us/step - loss: 0.8767 - acc: 0.6333
Epoch 6/10
100000/100000 [==============================] - 100s 996us/step - loss: 0.8453 - acc: 0.6468
Epoch 7/10
100000/100000 [==============================] - 102s 1ms/step - loss: 0.8157 - acc: 0.6586
Epoch 8/10
100000/100000 [==============================] - 101s 1ms/step - loss: 0.7834 - acc: 0.6718
Epoch 9/10
100000/100000 [==============================] - 103s 1ms/step - loss: 0.7501 - acc: 0.6858
Epoch 10/10
10000/10000 [==============================] - 1s 124us/

In [ ]:
# predicting
test_pre = model.predict(test_data_matrix, batch_size=batch_size)
sub_df = pd.DataFrame()
sub_df["review_id"] = test_id_list
sub_df["pre"] = test_pre
sub_df.to_csv("pre.csv", index=False)